In [2]:
from transformers import AutoTokenizer, AutoModel

# 加载原始 tokenizer 和模型
model_name = "sentence-transformers/all-distilroberta-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 常见的 LaTeX 数学符号和命令（可以根据需要扩展）
latex_tokens = [
    # 基本运算符
    "\\frac", "\\sqrt", "\\cdot", "\\times", "\\div", "\\pm", "\\mp", "\\ast", "\\star",

    # 关系符
    "\\leq", "\\geq", "\\neq", "\\approx", "\\equiv", "\\sim", "\\propto",

    # 箭头符号
    "\\rightarrow", "\\leftarrow", "\\Rightarrow", "\\Leftarrow", "\\leftrightarrow",

    # 极限与积分
    "\\sum", "\\prod", "\\int", "\\lim", "\\infty", "\\partial", "\\nabla", "\\oint",

    # 集合与逻辑
    "\\in", "\\notin", "\\subset", "\\supset", "\\subseteq", "\\supseteq",
    "\\cup", "\\cap", "\\exists", "\\forall", "\\neg", "\\land", "\\lor",

    # 函数与运算符
    "\\sin", "\\cos", "\\tan", "\\csc", "\\sec", "\\cot",
    "\\log", "\\ln", "\\exp",

    # 希腊字母（小写）
    "\\alpha", "\\beta", "\\gamma", "\\delta", "\\epsilon", "\\zeta", "\\eta",
    "\\theta", "\\iota", "\\kappa", "\\lambda", "\\mu", "\\nu", "\\xi",
    "\\pi", "\\rho", "\\sigma", "\\tau", "\\upsilon", "\\phi", "\\chi", "\\psi", "\\omega",

    # 希腊字母（大写）
    "\\Gamma", "\\Delta", "\\Theta", "\\Lambda", "\\Xi", "\\Pi", "\\Sigma", "\\Phi", "\\Psi", "\\Omega",

    # 括号
    "\\left(", "\\right)", "\\left[", "\\right]", "\\left\\{", "\\right\\}",

    # 其他
    "\\dots", "\\ldots", "\\cdots", "\\vdots", "\\ddots",
    "\\text", "\\mathrm", "\\mathbb", "\\mathbf", "\\mathcal",
]

# 添加 token
num_added = tokenizer.add_tokens(latex_tokens)
print(f"共添加了 {num_added} 个 LaTeX token")

# 调整模型的 embedding 层
model.resize_token_embeddings(len(tokenizer))
print("模型 embedding 已调整为新词表大小。")


c:\Anaconda3\envs\main\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


共添加了 100 个 LaTeX token
模型 embedding 已调整为新词表大小。


In [13]:
from sentence_transformers import InputExample
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("./init_latex_augmented_model")
org_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-distilroberta-v1")
tokenizer = AutoTokenizer.from_pretrained("./init_latex_tokenizer")

ss = "\\frac{a}{b} + \\sqrt{x}"
# print(model(org_tokenizer(ss, return_tensors="pt")))
input = tokenizer(ss, return_tensors="pt")
smodel = SentenceTransformer("./init_latex_augmented_model")
print(smodel.encode(["\\frac{a}{b} + \\sqrt{x}"]))

No sentence-transformers model found with name ./init_latex_augmented_model. Creating a new one with mean pooling.


[[ 4.80992049e-01 -2.11796597e-01  2.25521013e-01 -5.33645935e-02
  -5.57615697e-01 -2.07341030e-01  2.89255917e-01  4.09218311e-01
   1.79379374e-01 -2.95992792e-01 -5.86548030e-01  4.55772178e-03
   2.17057943e-01  2.14373320e-01 -2.03988835e-01 -5.77347398e-01
   5.11381984e-01 -3.91437635e-02  3.74132156e-01  9.81906891e-01
  -3.27287875e-02  2.48191819e-01  7.69005358e-01  1.81317955e-01
   5.21607518e-01  2.90802985e-01  9.60595012e-01  4.67844784e-01
  -7.34464705e-01 -3.23644489e-01 -1.48589388e-01  1.62972331e-01
  -2.04809502e-01 -2.33475581e-01 -1.45799994e-01  7.16955066e-02
  -5.97023889e-02 -6.30998984e-02  3.88231248e-01  6.70483291e-01
   4.44551766e-01  5.97410738e-01 -4.77533400e-01  3.28371614e-01
   3.00591141e-01  1.05956113e+00 -2.45312467e-01 -1.25858590e-01
  -3.58625621e-01 -4.23336655e-01 -7.60289192e-01  5.26056945e-01
  -5.40275156e-01  8.29386353e-01  2.41581365e-01  6.88870549e-02
  -4.49270099e-01 -6.94311023e-01  8.76966119e-02 -2.23104730e-01
   2.98613